## Pre-Processing_Dataset_1.py
Ana Cismaru

In [4]:
import os
import numpy as np
import mne
import cloudstorage as gcs

ModuleNotFoundError: No module named 'cloudstorage'

This notebook uses the Sleep Cassette Study (153 recordings in .edf format) and Data from Sleep EDF Expanded Dataset
Link: https://physionet.org/content/sleep-edfx/1.0.0/

In [5]:
#GCP stuff
from google.appengine.api import app_identity

bucket_name = os.environ.get('sleep-edfx-1.0.0.physionet.org',
                               app_identity.get_default_gcs_bucket_name())

ModuleNotFoundError: No module named 'google'

<b> Upload Data </b>

Input: String that contains folder name of folder with PSG.edf files


Output: list of MNE Raw Objects

In [ ]:
def __main__(data_folder):
    for path in os.listdir(data_folder):
        if path.endswith("PSG.edf"):
            full_path = os.path.join(data_folder, path)
            raw = mne.io.read_raw_edf(full_path, preload=True)
            mne_eeg = remove_sleepEDF(raw)
            mne_filtered = filter(mne_eeg)
            save(mne_filtered, path[:path.index("-")])

<b> Remove Non EEG Channels </b>

Input: list of MNE Raw Objects

Output: list of MNE Raw Objects with only EEG channels

In [ ]:
#For sleep edf, manual extraction because mne.pick_types isn't working

def remove_sleepEDF(mne_raw):
    return mne_raw.pick_channels(["EEG Fpz-Cz", "EEG Pz-Oz"])

In [ ]:
#In general
def remove(mne_raw):
    mne_eeg_gen = []
    for point in mne_raw:
        mne_eeg_gen.append(point.pick_types(eeg = True))
    return mne_eeg_gen

<b> Filter 30 HZ 4th-order FIR lowpass filter </b>

Input: list of MNE Raw Objects with only EEG chaneels

Output: list of filtered MNE Raw Objects


In [ ]:
def filter(mne_eeg):
    point2 = mne_eeg.filter(l_freq=None,
            h_freq=30,
            picks = ["EEG Fpz-Cz", "EEG Pz-Oz"],
            filter_length = "auto",
            method = "fir"
            )
    return point2

<b> Pickle Objects </b>

Input: A list of preprocessed MNE Objects.
    
Output: A FIF file

In [ ]:
import pickle

with open('data1.pkl', 'wb') as f:
    pickle.dump(mne_filtered, f)

<b> Save as FIF files </b>

Input: A list of preprocessed MNE Objects
    
Output: A FIF file for each datapoint

In [ ]:
def save(mne_filtered, name):
    mne_filtered.save("data1/data1_{num}_raw.fif".format(num = name))

<b> Run pipeline </b>

In [ ]:
data_folder = "sleep-cassette"
__main__(data_folder)

<b> Label Engineering: merge labels 3 and 4 </b>

Input: path to folder containing Hypnogram label files (.edf format)

Output: list of MNE Annotation Objects

In [ ]:
data_folder = "sample_hypnogram_data"

def upload_labels(data_folder):
    labels = []
    for path in os.listdir(data_folder):
        if path.endswith(".edf"):
            full_path = os.path.join(data_folder, path)
            label = mne.read_annotations(full_path)
            labels.append(label)
    return labels

labels = upload_labels(data_folder)

Input: list of MNE Annotation Objects

Output: list of MNE Annotation Objects with merged labels 3 and 4 --> now label 3

In [ ]:
def merge(labels):
    
    def merge_labels(label):
        new_label = np.array([])
        for stage in label:
            if stage == "Sleep stage 4":
                new_label = np.append(new_label, "Sleep stage 3")
            else:
                new_label = np.append(new_label, stage)
        return new_label
    
    for label in labels:
        label.description = merge_labels(label.description)
    
merge(labels)        

In [ ]:
raw = mne.io.read_raw_fif(full_path, preload=True)